# **Validación del modelo BERT para extracción de entidades médicas en historias clínicas de cáncer de pulmón**

Presentado por:

* Mayra Erazo
* Yeraldin Tafur
* Roberto Ceballos
* Katheryn Sanchez

A continuación, realizamos la validación del modelo construido (Mayra13/bert-base-uncased-finetuned-ner-pulmon) para predecir etiquetas de historias clínicas de pacientes con cáncer de pulmón.

# **1. Instalación de Librerías**
Se instalan e importan las librerías necesarias para el procesamiento del lenguaje natural, uso del modelo BERT y manejo de datos.

In [1]:
!pip install transformers[torch]
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from huggingface_hub import login
login(getpass("Introduce tu token de Hugging Face: ")) # Token de acceso personal a Hugging Face

In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import torch.nn.functional as F
from tqdm import tqdm


# **2. Definición de etiquetas**
Se define el diccionario id2label que contiene las entidades médicas que el modelo está entrenado para reconocer, como conceptos de cáncer, tratamientos, fechas, entre otros.




In [4]:
### Diccionario con las etiquetas usadas en el modelo
id2label = {
    0: 'B_CANCER_CONCEPT',
    1: 'B_CHEMOTHERAPY',
    2: 'B_DATE',
    3: 'B_DRUG',
    4: 'B_FAMILY',
    5: 'B_FREQ',
    6: 'B_IMPLICIT_DATE',
    7: 'B_INTERVAL',
    8: 'B_METRIC',
    9: 'B_OCURRENCE_EVENT',
    10: 'B_QUANTITY',
    11: 'B_RADIOTHERAPY',
    12: 'B_SMOKER_STATUS',
    13: 'B_STAGE',
    14: 'B_SURGERY',
    15: 'B_TNM',
    16: 'I_CANCER_CONCEPT',
    17: 'I_DATE',
    18: 'I_DRUG',
    19: 'I_FAMILY',
    20: 'I_FREQ',
    21: 'I_IMPLICIT_DATE',
    22: 'I_INTERVAL',
    23: 'I_METRIC',
    24: 'I_OCURRENCE_EVENT',
    25: 'I_SMOKER_STATUS',
    26: 'I_STAGE',
    27: 'I_SURGERY',
    28: 'I_TNM',
    29: 'O'
}

num_labels = len(id2label)



# **3. Carga del modelo y del tokenizer**
Se carga el modelo fine-tuned desde Hugging Face junto con su tokenizer.

In [5]:
# Cargar modelo y tokenizer
# Se carga el modelo entrenado previamente
hugging_face_NER_model="Mayra13/bert-base-uncased-finetuned-ner-pulmon"

model = AutoModelForTokenClassification.from_pretrained(hugging_face_NER_model,
        num_labels = num_labels,
        id2label = id2label,
        label2id = {v: k for k, v in id2label.items()}
)

tokenizer = AutoTokenizer.from_pretrained(hugging_face_NER_model, use_fast = True)


# Usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



all_results = []
batch_size = 8


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

# **4. Textos de prueba**
Se define una lista de frases representativas de historias clínicas relacionadas con cáncer de pulmón que se usarán para validar el modelo.


In [6]:
texts = ["Historia oncológica: Carcinoma escamoso de pulmón cT2a cN2 cM1c estadio IVB por afectación hepática y ósea.",
         "Varon de 75 años diagnosticado con Carcinoma neuroendocrino de célula grande de pulmón c T4 N2 M1 estadio IV (por afectacion renal)",
         "Paciente diagnosticado con carcinoma pulmonar de células no pequeñas en estadio IIIA.",
         "Se inició tratamiento con quimioterapia basada en cisplatino y etopósido.",
         "La cirugía torácica fue realizada el 12 de marzo de 2023 sin complicaciones.",
         "Fumador activo, consume 20 cigarrillos al día desde hace 30 años.",
         "Se identificó un adenocarcinoma broncoalveolar con metástasis en ganglios linfáticos.",
         "Radioterapia administrada durante un intervalo de seis semanas consecutivas.",
         "La masa pulmonar mide 3.5 cm de diámetro en el lóbulo inferior derecho.",
         "Hermano del paciente con antecedente de carcinoma pulmonar.",
         "Se programó la próxima quimioterapia para dentro de dos semanas.",
         "Clasificación TNM reportada como T2 N1 M0 en el último control."
        ]


# **5. Tokenización de las oraciones**
Convierte las frases en tokens compatibles con el modelo BERT (incluye máscara de atención, padding, truncamiento, etc.).

In [7]:
# Tokenización
encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        return_offsets_mapping=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        max_length=512,
        is_split_into_words=False
        )

# **6. Predicciones**
Una vez tokenizadas, las frases se pasan por el modelo y se obtienen las predicciones.
Calculamos las probabilidades usando softmax y tomamos la etiqueta con mayor score para cada token.

In [8]:
input_ids = torch.tensor(encodings["input_ids"]).to(device)

attention_mask = torch.tensor(encodings["attention_mask"]).to(device)


with torch.no_grad():
 outputs = model(input_ids=input_ids, attention_mask=attention_mask)

logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)
probs = F.softmax(logits, dim=-1)

Observamos el código de la etiqueta predicha para cada token.

In [9]:
print (predictions)


tensor([[29, 29, 29, 29, 29, 29, 29,  0,  0, 16, 16, 16, 16, 16, 16, 16, 16, 15,
         28, 28, 28, 28, 28, 28, 28, 13, 26, 26, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29],
        [29, 29, 29, 29, 10,  8, 29,  9, 29, 29,  0,  0, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 28, 28, 28, 28, 28, 13, 26, 29,
         29, 29, 29, 29, 29, 29, 29],
        [29, 29, 29,  9, 29, 29,  0,  0, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 29, 13, 26, 26, 29, 29, 29,  9, 29, 29, 29, 29,  9, 29,  0,
         16, 16, 16, 16, 16, 16, 16],
        [29, 29,  9, 24, 24, 24, 24, 29,  1, 29, 29, 29, 29, 29, 29, 29,  3,  3,
          3, 18, 29,  3,  3,  5, 29, 29, 29,  9,  9,  9,  9, 29,  9, 29, 29, 29,
         29, 29, 29, 29,  9,  3, 29],
        [29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  2, 17, 17, 17,
         17, 17, 17, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
         29, 29, 29, 29, 29, 29, 29],


# **7. Alineación de tokens con etiquetas**
Se transforman los subtokens en palabras completas y se alinean con sus etiquetas y puntuaciones de confianza (scores).

In [29]:
### Para cada oracion en la lista de oraciones.
aligned_results = []
for i, text in enumerate(texts):
    word_ids = encodings.word_ids(batch_index=i)
    tokens = tokenizer.convert_ids_to_tokens(encodings["input_ids"][i])

    previous_word_id = None
    aligned_words, aligned_labels, aligned_scores = [], [], []

    for j, (token, label_id, word_id) in enumerate(zip(tokens, predictions[i].tolist(), word_ids)):
        if word_id is None:
            continue

        # Probabilidad del label predicho para este token
        prob = probs[i][j][label_id].item()

        token_clean = token.replace("▁", "").replace("##", "")
        if word_id != previous_word_id:
            aligned_words.append(token_clean)
            aligned_labels.append(id2label[label_id])
            aligned_scores.append(prob)
        else:
            aligned_words[-1] += token_clean
            # OJO: Si combinas subwords, toma el max de los scores
            aligned_scores[-1] = max(aligned_scores[-1], prob)

        previous_word_id = word_id

    filtered_results = [
        (word, label, score)
        for word, label, score in zip(aligned_words, aligned_labels, aligned_scores)
        if label != "O"
    ]

    aligned_results.append(filtered_results)

    ### Resultados por oración
    print(f"\n Resultados alineados para la oración {i+1}:")
    print("Palabras:", aligned_words)
    print("Labels:", aligned_labels)
    print("Scores:", aligned_scores)



 Resultados alineados para la oración 1:
Palabras: ['historia', 'oncologica', ':', 'carcinoma', 'escamoso', 'de', 'pulmon', 'ct2a', 'cn2', 'cm1c', 'estadio', 'ivb', 'por', 'afectacion', 'hepatica', 'y', 'osea', '.']
Labels: ['O', 'O', 'O', 'B_CANCER_CONCEPT', 'I_CANCER_CONCEPT', 'I_CANCER_CONCEPT', 'I_CANCER_CONCEPT', 'B_TNM', 'I_TNM', 'I_TNM', 'B_STAGE', 'I_STAGE', 'O', 'O', 'O', 'O', 'O', 'O']
Scores: [0.9999380111694336, 0.9999243021011353, 0.9999150037765503, 0.9997343420982361, 0.9995983242988586, 0.9996185302734375, 0.9996167421340942, 0.9977225661277771, 0.9986069798469543, 0.9986479878425598, 0.9991766810417175, 0.9989590644836426, 0.999937891960144, 0.9999339580535889, 0.9999388456344604, 0.999940037727356, 0.999937891960144, 0.9999300241470337]

 Resultados alineados para la oración 2:
Palabras: ['varon', 'de', '75', 'anos', 'diagnosticado', 'con', 'carcinoma', 'neuroendocrino', 'de', 'celula', 'grande', 'de', 'pulmon', 'c', 't4', 'n2', 'm1', 'estadio', 'iv', '(', 'por', 'af

En el código anterior dividimos cada palabra de cada oración por sus respectivos tokens, después se genera la predicción de la etiqueta utilizando el modelo con una confianza determinada, una vez generada la predicción de cada token, se unen los tokens para reconstruir las palabras para hacer legible y se une con la predicción y el score o confianza del modelo para predecir dicha etiqueta. Según los resultados podemos observar que muchas etiquetas para estas oraciones fueron predichas con una alta probabilidad por encima de 0.9, lo que hace que dichas predicciones sean confiables.

# **8. Unión de etiquetas tipo B-I y resultados del modelo**

Se realiza un proceso para unir los tokens etiquetados como comienzo ("B_") y continuación ("I_") de una misma entidad, de modo que cada concepto completo quede representado como una sola unidad (por ejemplo, “carcinoma escamoso de pulmón” en lugar de varias palabras separadas). Para cada entidad unificada, se calcula el promedio del score como medida de confianza del modelo para ese concepto. Finalmente, se imprime una lista ordenada con las entidades detectadas, sus tipos (como CANCER_CONCEPT, TNM, STAGE, etc.) y sus scores, y todo esto se guarda en una lista llamada all_results.

Para cada oración, primero se imprime el texto original, seguido de las palabras detectadas, sus etiquetas asignadas (usando el esquema BIO) y los puntajes de confianza (score) obtenidos por el modelo.

In [38]:
all_results = []

# Recorremos cada oración ya alineada con su resultado
for i, sentence_results in enumerate(aligned_results):
    print("=" * 100)
    print(f"\n Oración {i+1}")
    print("Texto:", texts[i])  # Mostramos la oración original

    # Extraemos los tokens, etiquetas y scores crudos
    tokens_crudos = [word for word, _, _ in sentence_results]
    labels_crudos = [label for _, label, _ in sentence_results]
    scores_crudos = [score for _, _, score in sentence_results]

    print("\nPalabras: ", tokens_crudos)
    print("Labels:  ", labels_crudos)
    print("Scores:  ", scores_crudos)

    print("\n **** Se unen las etiquetas B, I en una sola entidad **** \n")

    combined_results = []
    temp_entity, temp_label, temp_scores = "", "", []

    # Agrupamos entidades por etiquetas BIO
    for word, label, score in sentence_results:
        if label.startswith("B_"):
            if temp_entity:
                combined_results.append((temp_entity, temp_label, round(sum(temp_scores) / len(temp_scores), 6)))
            temp_entity = word
            temp_label = label[2:]  # Quitamos el prefijo B_
            temp_scores = [score]
        elif label.startswith("I_") and label[2:] == temp_label:
            temp_entity += " " + word
            temp_scores.append(score)
        else:
            if temp_entity:
                combined_results.append((temp_entity, temp_label, round(sum(temp_scores) / len(temp_scores), 6)))
            temp_entity, temp_label, temp_scores = "", "", []

    if temp_entity:
        combined_results.append((temp_entity, temp_label, round(sum(temp_scores) / len(temp_scores), 6)))

    # Imprimimos las entidades agrupadas
    for entity, label, score in combined_results:
        result = {
            "Palabra": entity,
            "Entidad": label,
            "Score": score
        }
        print(result)
        all_results.append(result)



 Oración 1
Texto: Historia oncológica: Carcinoma escamoso de pulmón cT2a cN2 cM1c estadio IVB por afectación hepática y ósea.

Palabras:  ['carcinoma', 'escamoso', 'de', 'pulmon', 'ct2a', 'cn2', 'cm1c', 'estadio', 'ivb']
Labels:   ['B_CANCER_CONCEPT', 'I_CANCER_CONCEPT', 'I_CANCER_CONCEPT', 'I_CANCER_CONCEPT', 'B_TNM', 'I_TNM', 'I_TNM', 'B_STAGE', 'I_STAGE']
Scores:   [0.9997343420982361, 0.9995983242988586, 0.9996185302734375, 0.9996167421340942, 0.9977225661277771, 0.9986069798469543, 0.9986479878425598, 0.9991766810417175, 0.9989590644836426]

 **** Se unen las etiquetas B, I en una sola entidad **** 

{'Palabra': 'carcinoma escamoso de pulmon', 'Entidad': 'CANCER_CONCEPT', 'Score': 0.999642}
{'Palabra': 'ct2a cn2 cm1c', 'Entidad': 'TNM', 'Score': 0.998326}
{'Palabra': 'estadio ivb', 'Entidad': 'STAGE', 'Score': 0.999068}

 Oración 2
Texto: Varon de 75 años diagnosticado con Carcinoma neuroendocrino de célula grande de pulmón c T4 N2 M1 estadio IV (por afectacion renal)

Palabras: 

# **Conclusiones**
Después de procesar las 12 oraciones clínicas, el modelo logró identificar correctamente un total de 27 entidades importantes relacionadas con el cáncer de pulmón, como tipos de cáncer, estadios, tratamientos y antecedentes.

En casi todos los casos, los scores de confianza fueron muy altos (casi todos cercanos a 1.0), lo que indica que el modelo está muy seguro de sus predicciones y que está reconociendo muy bien las entidades.

Las categorías más frecuentes que detectó fueron:

CANCER_CONCEPT: por ejemplo, “carcinoma escamoso de pulmón”

TNM: que indica la clasificación del tumor

STAGE: estadio de la enfermedad

DRUG y CHEMOTHERAPY: nombres de medicamentos y tratamientos

También reconoció eventos como fechas, consumo de cigarrillos y antecedentes familiares.

En resumen, el modelo funciona muy bien para este tipo de textos médicos en español y es útil para extraer información clave de forma automática.